In [1]:
%gui qt
%pylab qt
import numpy as np
import re
from pyknotid.spacecurves import Knot
from pyknotid.representations import GaussCode, Representation
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from tqdm import tnrange, tqdm_notebook
from time import sleep

Populating the interactive namespace from numpy and matplotlib


### Splitter 
A function for parsing PyKnotId gauss code in preparation for conversion.

In [2]:
def splitter(s):
    if '+' in s:
        
        sign = '+'
        number, hand = s.split('+')
    elif '-' in s:
        sign = '-'
        
        number, hand = s.split('-')
    return number, sign, hand            

### Gauss Converter
Puts the gauss code and extended gauss code into a form PyKnotId can work with

In [3]:

def gauss_converter(g,e):
    newgauss = ''
    for i in g:
        if i < 0:
            # c for clockwise
            # a for anticlockwise
            if e[abs(i)-1]==1:
                newgauss += str(abs(i))+'-c,'
            else:
                newgauss += str(abs(i))+'-a,'
        else:
            if e[abs(i)-1]==1:
                newgauss += str(abs(i))+'+c,'
            else:
                newgauss += str(abs(i))+'+a,'
    return newgauss[:-1]



### Gauss Reverter
Converts the gauss code from PyKnotId into a form we are familiar with

In [4]:
def gauss_reverter(g):
    numbers = []
    signs = []
    hands = []
    
    gauss = g.split(',')
    for i in gauss:
        number, sign, hand = splitter(i)
        numbers.append(int(number))
        signs.append(sign)
        hands.append(hand)
    ex_g_code = list(np.zeros(max(numbers),dtype=int))

    for i,j in enumerate(numbers):
        if signs[i] == '-':
            numbers[i] *= -1
        if ex_g_code[abs(j)-1] == 0:
            if hands[i] == 'a':
                ex_g_code[abs(j)-1] = -1
            else:
                ex_g_code[abs(j)-1] = 1
                

    return numbers, ex_g_code

### Get the Gauss Codes

In [5]:
def get_code(low=10,high=50,run=20,reparam=100000,interp_pts=400,pt_random = True,point_amt=0):
    if pt_random: 
        point_amt = np.random.randint(low,high)
    else:
        point_amt = point_amt 
    points = np.random.rand(point_amt,3)
    pm = lambda x:x*np.random.choice([-1,1])
    mult = np.array([np.random.randint(1,30) for i in range(point_amt)])
    for i,j in enumerate(points):
        if j[0]==j[1] and j[1]==j[2] and j[2] == 0:
            break
        points[i] = [pm(j) for j in points[i]]
        points[i] *= 1/np.sqrt(j[0]**2+j[1]**2+j[2]**2)
        points[i]*= point_amt

    gc=Knot(points).gauss_code() #Generate knot object from points tuple
    '''
    k.octree_simplify(runs = run)
    k=k.reparameterised(num_points = reparam)
    k.interpolate(interp_pts)
    k.plot() #Plot the knot in vispy

    k.plot_projection() # plot the projection in matplotlib
    '''
    gcs = gc.strang()

    if len(gc)>2:    
        gauss1, extend1 = gauss_reverter(gcs)
    #print("Code before:\n",gauss,extend)
        gc.simplify()
    else:
        gauss1 = [0]
        extend1 = [0]
    if len(gc)>2:
        gcs = gc.strang()
        gauss2, extend2 = gauss_reverter(gcs)
    else:
        gauss2 = [0]
        extend2 = [0]
    #print("\nCode after: ",gauss,extend)
    gc1 = str(gauss1)[1:-1]+'/'+str(extend1)[1:-1]
    gc2 = str(gauss2)[1:-1]+':'+str(extend2)[1:-1]
    total_gauss = gc1+'*'+gc2
    return total_gauss

In [6]:
'''file = open("GaussCodes.txt","w")
loops = 55
for i in tnrange(5,5+loops, desc='1st loop'):
    for j in range(1000):
      
        total_gauss = get_code(low=i,high=i+40,run=20,reparam=100000,interp_pts=400,pt_random = False,point_amt=i)
        if total_gauss == '0/0*0:0':
            continue
        else:
            file.write(str(total_gauss)+'\n')
            loop.set_description('On Point:{}'.format(i))
        
    
file.close()
'''

'file = open("GaussCodes.txt","w")\nloops = 55\nfor i in tnrange(5,5+loops, desc=\'1st loop\'):\n    for j in range(1000):\n      \n        total_gauss = get_code(low=i,high=i+40,run=20,reparam=100000,interp_pts=400,pt_random = False,point_amt=i)\n        if total_gauss == \'0/0*0:0\':\n            continue\n        else:\n            file.write(str(total_gauss)+\'\n\')\n            loop.set_description(\'On Point:{}\'.format(i))\n        \n    \nfile.close()\n'

HI
